<a href="https://colab.research.google.com/github/SophiaU/Git_DoD_Remittance/blob/main/Remittance_Weekly_Reconciliation_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install and import pygsheets
!pip install pygsheets
import pygsheets

#  Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# connect google sheets
import gspread
gc = gspread.authorize(GoogleCredentials.get_application_default())

from google.colab import files
# uploaded = files.upload()

import json
from google.oauth2 import service_account

# with open('service_account.json') as source:
#    info = json.load(source)
# credentials = service_account.Credentials.from_service_account_info(info)

# client = pygsheets.authorize(service_account_file='service_account.json')

# import pandas
import pandas as pd
from functools import reduce

# making a change here

     |████████████████████████████████| 153kB 5.3MB/s 


Combine Performance File

In [ ]:
# Append all files in folder into a listframe

listframe = []

weekname = 'Jul 05'
fileid = '1622D4YEaxFgT_6oGCPesUReORnys57jD'

folder = drive.ListFile({'q': "title contains '"+weekname+"' and not title contains '(7)' and not title contains '(6)' and '"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
for file in folder:
  print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('performance.csv')

  # read
  df = pd.read_csv('performance.csv', delimiter=',' )

  # append
  listframe.append(df)

# combine all files in the list into a dataframe
wkperffile = pd.concat(listframe)

# pick only name, trips and supply hours
performancefile = wkperffile[['Name','Trips','Hours online']]

# capitalize the name column
performancefile['Name'] = pd.Series(performancefile['Name']).str.title()

# convert hour online to just hours
performancefile['Hours online'] = pd.to_timedelta(performancefile['Hours online']).dt.total_seconds()/3600

# sum trips and hours by names
performancefile = performancefile.groupby(['Name']).sum().reset_index()

print(performancefile)

driver_performance Jul 05(5).csv
driver_performance Jul 05(4).csv
driver_performance Jul 05(2).csv
driver_performance Jul 05(3).csv
driver_performance Jul 05(1).csv
                            Name  Trips  Hours online
0                 Abayomi Ayinde     29          48.5
1               Abimbola Omojuwa     63          69.9
2                Abiodun Adedeji     49          63.2
3                Abiodun Onamosu     53          57.5
4           Abiodun Sunday Adisa     56          60.7
..                           ...    ...           ...
347          Williams Aikhuemero     71          89.8
348  Williams Nwadinobi Anunukem     48          77.7
349                 Wisdom Akpan     56          65.0
350     Yetunde Samiat Shobowale     12          12.9
351   Zainab Oluwasefunmi Atuaka     32          33.4

[352 rows x 3 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Combine Payments file

In [ ]:
#  Append all files in folder into a listframe
paymentslistframe = []

fileid = '15ylXHnFS3bh-2IbSboSrGbCLL3zW6bj9'

folder = drive.ListFile({'q': "title contains '"+weekname+"' and not title contains '(7)' and not title contains '(6)' and '"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()
for file in folder:
  print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('payments.csv')

  # read
  df = pd.read_csv('payments.csv', delimiter=',' )

  # append
  paymentslistframe.append(df)

# combine all files in the list into a dataframe
combinedPayments = pd.concat(paymentslistframe)

# concatenate the names
combinedPayments['Name'] = combinedPayments['firstName'] + " " + combinedPayments['lastName']
combinedPayments['Name'] = pd.Series(combinedPayments['Name']).str.title()
combinedPayments = combinedPayments[['Name','driverUUID', 'tripUUID', 'firstName', 'lastName', 'amount','timestamp', 'itemType', 'description', 'disclaimer']]

# pick only fullname, amount and itemtype
paymentsfile = combinedPayments[['Name','driverUUID','amount','itemType']]

# get net earnings
# drop cash collected itemtype
netEarnings = paymentsfile.loc[paymentsfile['itemType'] != 'cash_collected']

# drop payouts itemtype
netEarnings = netEarnings.loc[netEarnings['itemType'] != 'payouts']

# sum amounts by names to get the net earnings per driver
netEarnings = netEarnings.groupby(['driverUUID']).sum().reset_index()
netEarnings.columns = ['driverUUID','Net Earnings']
# netEarnings['Name'] = pd.Series(netEarnings['Name']).str.title()
netEarnings.head(400)

# get total cash collected
# get cash collected itemtype
cashCollected = paymentsfile.loc[paymentsfile['itemType'] == 'cash_collected']

# sum amounts by names to get the cash collected per driver
cashCollected = cashCollected.groupby(['driverUUID']).sum().reset_index()
cashCollected.columns = ['driverUUID', 'Cash Collected']
#cashCollected['Name'] = pd.Series(cashCollected['Name']).str.title()

cashCollected.head(400)

Payments Jul 05(2).csv
Payments Jul 05(3).csv
Payments Jul 05(4).csv
Payments Jul 05(5).csv
Payments Jul 05(1).csv


,driverUUID,Cash Collected
0,001319fb-c6f6-4849-a062-515e59dc0689,-60900.0
1,0073465c-8143-49c6-a37a-4f0b29738c05,-62100.0
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,-47200.0
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,-95200.0
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,-28500.0
...,...,...
348,fa140fd5-34a0-447a-a581-9e897516c4d9,-79700.0
349,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,-80100.0
350,fb49c327-2bac-4479-8866-fad76b89e207,-108100.0
351,fce722ca-e6db-456e-842d-ea22857d4cb1,-35600.0


In [ ]:
#Add Driver Uber ID to Performance file

uniqueID = combinedPayments.groupby(['Name','driverUUID']).sum().reset_index()

# uniqueID = combinedPayments.drop_duplicates(
#   subset = ['Name', 'driverUUID'],
#   keep = 'last').reset_index(drop = True)

performancefile1 = pd.merge(performancefile, uniqueID[['Name','driverUUID']], on='Name', how='left')
performancefile1 = performancefile1[['Name','Trips','Hours online','driverUUID']]
performancefile1 = performancefile1.groupby(['driverUUID']).sum().reset_index()
performancefile1


# NameCheck = performancefile1.query('driverUUID == "de90e988-5c1d-4ee7-ae7d-0a1418cdd111"')
# NameCheck = performancefile1.query('Name == "Onyeka Aigbaze"')        # Isiaq Jamiu Akanbi
# NameCheck

,driverUUID,Trips,Hours online
0,001319fb-c6f6-4849-a062-515e59dc0689,41,53.0
1,0073465c-8143-49c6-a37a-4f0b29738c05,50,57.2
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,42,37.2
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,89,88.6
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,34,35.4
...,...,...,...
347,fa140fd5-34a0-447a-a581-9e897516c4d9,48,77.7
348,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,58,73.3
349,fb49c327-2bac-4479-8866-fad76b89e207,90,75.7
350,fce722ca-e6db-456e-842d-ea22857d4cb1,18,25.2


In [ ]:
netEarnings.head(400)

,driverUUID,Net Earnings
0,001319fb-c6f6-4849-a062-515e59dc0689,70659.75
1,0073465c-8143-49c6-a37a-4f0b29738c05,68632.75
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,53458.60
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,123862.75
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,40900.75
...,...,...
349,fa140fd5-34a0-447a-a581-9e897516c4d9,89450.25
350,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,80205.35
351,fb49c327-2bac-4479-8866-fad76b89e207,132843.50
352,fce722ca-e6db-456e-842d-ea22857d4cb1,31575.75


In [ ]:
# # export to csv
# netearnings_csv = netEarnings.to_csv( "netearnings.csv", index=False, encoding='utf-8-sig')

# file2 = drive.CreateFile({'parents': [{'id': '15w_MeKLpCE2hs8-Vvs4IUBH_sZ8e3qbP'}]})
# file2.SetContentFile("netearnings.csv")
# file2.Upload()

In [ ]:
# Write combined payments file to google sheets
# combinedPayments = combinedPayments[['Name','driverUUID', 'tripUUID', 'firstName', 'lastName', 'amount','timestamp', 'itemType', 'description', 'disclaimer']]
# combinedPayments = combinedPayments.fillna("")
# dtoSheet = client.open_by_key('1cxkEP4YqeVnQ4guhn8D-NMUxJdH4YRF7zX9v9IV3WhM')
# wkspaymentfile = dtoSheet[1]

# wkspaymentfile.set_dataframe(combinedPayments,(1,2))

In [ ]:
performancefile1.head(400)

,driverUUID,Trips,Hours online
0,001319fb-c6f6-4849-a062-515e59dc0689,41,53.0
1,0073465c-8143-49c6-a37a-4f0b29738c05,50,57.2
2,0078ddf7-9f70-4fc1-bf93-80a5114dfbfc,42,37.2
3,011bfaae-ef9c-40c4-b49a-5acfc3f5e56c,89,88.6
4,01ef8975-0e8c-41a1-982e-f3ded75449a5,34,35.4
...,...,...,...
347,fa140fd5-34a0-447a-a581-9e897516c4d9,48,77.7
348,fb2060b2-b6a4-4e59-b411-a6fa2c5623a0,58,73.3
349,fb49c327-2bac-4479-8866-fad76b89e207,90,75.7
350,fce722ca-e6db-456e-842d-ea22857d4cb1,18,25.2


In [ ]:
# performance_csv = performancefile.to_csv( "performance.csv", index=False, encoding='utf-8-sig')

# file2 = drive.CreateFile({'parents': [{'id': '15w_MeKLpCE2hs8-Vvs4IUBH_sZ8e3qbP'}]})
# file2.SetContentFile("performance.csv")
# file2.Upload()

Get Moove Details

In [ ]:
# get moove details
mooveDetailList =[]
worksheet = gc.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg').sheet1
for i in range(2,6):
  columns = worksheet.col_values(i)
  mooveDetailList.append(columns[1:])

# print(mooveDetailList)

# convert list to data frame
mooveDetails = pd.DataFrame(mooveDetailList)
mooveDetails = mooveDetails.T
mooveDetails.columns = ['Name','Product','DRN','driverUUID']
mooveDetails['Name'] = pd.Series(mooveDetails['Name']).str.title()
#print(mooveDetails)
mooveDetails

,Name,Product,DRN,driverUUID
0,Inwang Ubong,income product,DRN000022,4ed6faed-e165-4a8b-8e47-9a24433ddb91
1,Adeola Okusaga,income product,DRN000220,b2e4d5a7-fbae-49a2-be3a-4ba1b9f179f5
2,Temitope Tayo,income product,DRN000229,
3,Chiegeokwu Amaju,flexi rental,DRN000230,0f23e379-8b69-4233-9497-0ad579a2f60a
4,Adanna Cythia Nwanna,flexi rental,DRN000233,0af65713-855a-4a7f-8d15-5fca9dbfecbb
...,...,...,...,...
930,Paul Akere Omokaro,DTO-II 48,DRN002663,
931,Kelechi Oluwatoba Okorie,DTO-II 48,DRN002662,ba59a846-4044-4fae-84c6-81a9628fd6b7
932,Omololu Kehinde Sotonwa,DTO-II 48,DRN002633,a1660b79-4752-4dfd-a79b-3bc14db3163d
933,Aimuanmwosa Amos Asemota,DTO-II 48,DRN002654,


Virtual Accounts Data

In [ ]:
fileid = '18oe6VePd_c0cqPkrXI8QfvHixpECOn_Z'

folder = drive.ListFile({'q': "title contains '"+weekname+"' and not title contains '(7)' and not title contains '(6)' and '"+fileid+"' in parents and mimeType = 'text/csv' and trashed = false "}).GetList()

for file in folder:
  print(file['title'])

  # download
  fileDownloaded = drive.CreateFile({'id':file['id']})

  # load
  fileDownloaded.GetContentFile('VirtualAccount.csv')

  # read
  VirtualAccountDF = pd.read_csv('VirtualAccount.csv', delimiter=',' )

# select only drn and amount
VirtualAcctDF = VirtualAccountDF[['drn','amount']]  
VirtualAcctDF.rename(columns={'drn':'DRN'}, inplace=True)

# Sum Amount
sumVA = VirtualAcctDF.groupby(['DRN']).sum().reset_index()
sumVA.rename(columns={'amount':'VA Collections'}, inplace=True)

# count amount
countVA = VirtualAcctDF.groupby(['DRN']).count().reset_index()
countVA.rename(columns={'amount':'VA Transactions'}, inplace=True)

print(sumVA)
print(countVA)

virtualAccount Jul 05.csv
          DRN  VA Collections
0   DRN000443             900
1   DRN000554           11200
2   DRN000574            2520
3   DRN000665            9300
4   DRN000762            5200
5   DRN001060            2052
6   DRN001093             500
7   DRN001112            9400
8   DRN001186           12400
9   DRN001218           14300
10  DRN001312            1700
11  DRN001649             500
12  DRN001708            5000
13  DRN001751            1000
14  DRN001893           11300
15  DRN002089            7800
          DRN  VA Transactions
0   DRN000443                1
1   DRN000554                3
2   DRN000574                5
3   DRN000665                6
4   DRN000762                4
5   DRN001060               31
6   DRN001093                1
7   DRN001112                5
8   DRN001186               16
9   DRN001218                7
10  DRN001312                1
11  DRN001649                1
12  DRN001708                4
13  DRN001751                1

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
# write va to google sheets

# reconSheet = client.open_by_key('1hWr33xeKPsgjA5oRjADCFHtH3CkeiebdJ1eMM6IRia0')
# wksreconfile = reconSheet[3]

# wksreconfile.set_dataframe(sumVA,(1,1))
# wksreconfile.set_dataframe(countVA,(1,4))

Days with car Data

In [ ]:
# get moove details
daysWithCarlList =[]
carsWorksheet = gc.open_by_key('1_hFrUFt5qoXbah9tLyQLnbmTZFPzeySYGYBnPjE3eZQ').sheet1
for i in range(10,12):
  columns = carsWorksheet.col_values(i)
  daysWithCarlList.append(columns[1:])

# print(mooveDetailList)

# convert list to data frame
daysWithCar = pd.DataFrame(daysWithCarlList)
daysWithCar = daysWithCar.T
daysWithCar.columns = ['DRN','Effective Days']
daysWithCar['DRN'] = pd.Series(daysWithCar['DRN']).str.strip()
daysWithCar = pd.merge(daysWithCar,mooveDetails,how='left', on='DRN')
# daysWithCar = daysWithCar.dropna()

daysWithCar


,DRN,Effective Days,Name,Product,driverUUID
0,DRN001222,1,Ifeanyichukwu Obi,income product,e403396e-4638-42f6-b116-ea811a998d96
1,DRN000385,2,Henry Robinson,income product,667d3184-c76a-4335-8b80-2a4681c19876
2,DRN001758,1,Moses Okafor Oliver,DTO 30,4e6ab8d2-8603-47e0-b465-f60edce2f128
3,DRN001693,3,Niyi Aderonunmu,DTO 30,c6ea912c-f8e0-4c38-b38e-eecd4a2acfdb
4,DRN000504,2,Bright Izuchukwu Nzerem,DTO 30,163b845e-81d7-4207-9f77-bce83c5caf43
...,...,...,...,...,...
360,DRN002503,-2,Samuel Oluwatemilorun Adedigba,DTO-II 48,
361,DRN002652,-2,NaN,NaN,NaN
362,DRN002296,-2,Tomisin Korede Adebiyi,DTO-II 48,
363,DRN002658,-2,NaN,NaN,NaN


In [ ]:
# write va to google sheets

# reconSheet = client.open_by_key('1hWr33xeKPsgjA5oRjADCFHtH3CkeiebdJ1eMM6IRia0')
# wksreconfile = reconSheet[3]

# wksreconfile.set_dataframe(daysWithCar,(1,7))

Combine Performance, Earnings, Days with Car and Moove Details

In [ ]:
# join elements of the final payments file
dfs = [performancefile1,netEarnings, cashCollected]

# join on driverUUID
reconfile = reduce(lambda left,right: pd.merge(left,right,how='outer', on='driverUUID'), dfs)
reconfile = reconfile.fillna(0)

dfs = [reconfile,daysWithCar]
reconfile = reduce(lambda left,right: pd.merge(left,right,how='outer', on='driverUUID'), dfs)

# join on DRN
dfs = [reconfile,sumVA, countVA]
reconfile = reduce(lambda left,right: pd.merge(left,right,how='left', on='DRN'), dfs)

reconfile['Cash Collected'] = -reconfile['Cash Collected'].fillna(0)
reconfile['Uber Balance'] = reconfile['Net Earnings'] - reconfile['Cash Collected']
reconfile['Effective Days'] = reconfile['Effective Days'].fillna("Missing Car Record")
reconfile['Hours online'] = reconfile['Hours online'].fillna("Did not work")
reconfile['Trips'] = reconfile['Trips'].fillna(0)
reconfile['VA Transactions'] = reconfile['VA Transactions'].fillna(0).astype(int)
reconfile['VA Collections'] = reconfile['VA Collections'].fillna(0)
reconfile['VA Incentive'] = pd.DataFrame(map(lambda x: 7000 if x >= 20 else x * 100, reconfile['VA Transactions']))

reconfile = reconfile[['DRN', 'Name', 'Product', 'Uber Balance', 'Cash Collected', 'VA Collections', 'Net Earnings', 
        'Trips', 'Hours online', 'VA Transactions','VA Incentive','Effective Days','driverUUID']]

reconfile = reconfile.fillna('')
reconfile = reconfile.sort_values('Trips',ascending=False)
reconfile.head(400)


,DRN,Name,Product,Uber Balance,Cash Collected,VA Collections,Net Earnings,Trips,Hours online,VA Transactions,VA Incentive,Effective Days,driverUUID
50,DRN001232,Precious Jimmy Willoby,income product,27089.5,78200.0,0.0,105290,99.0,68.6,0,0,2,27922035-6a3f-4719-8014-5da7e24d83ae
263,DRN000702,Chuma Raymond,DTO 30,15607.2,131600.0,0.0,147207,94.0,95.6,0,0,2,b8c5f9e7-dc0b-4da7-8314-f66d56b31b63
193,DRN000655,Chukwuebuka Bright Mmaduakolam,DTO 48,15749,102200.0,0.0,117949,92.0,75.3,0,0,2,8393452f-5128-412b-9cad-6ddc80ed51a4
93,DRN001140,Ethelbert Joseph Oparah,DTO 30,36206.7,90000.0,0.0,126207,91.0,91,0,0,2,438d010d-0998-45a9-8d74-e06949b5e493
349,DRN000558,Taiwo Bamisaiye,income product,24743.5,108100.0,0.0,132844,90.0,75.7,0,0,2,fb49c327-2bac-4479-8866-fad76b89e207
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,DRN001144,Lateef Olayiwola Akinwande,income product,,-0.0,0.0,,0.0,Did not work,0,0,2,1246d5e0-3933-471d-a7dc-a2d62debe9c2
370,DRN001688,Oluwadamilare Olamide Osun,income product,,-0.0,0.0,,0.0,Did not work,0,0,2,a915cf1b-2926-4d69-ac86-02d1102eefa7
371,DRN000797,Olalekan Lawal Bello,DTO 48,,-0.0,0.0,,0.0,Did not work,0,0,2,fbbb9e0a-dd81-4778-866d-fc162755f26f
372,DRN002622,Aniete Bassey,DTO-II 48,,-0.0,0.0,,0.0,Did not work,0,0,2,c376c46a-c423-485a-8693-b55178e9a594


In [ ]:
# # export recon file to csv
reconfile_csv = reconfile.to_csv( "reconfile_TestFinal.csv", index=False, encoding='utf-8-sig')
# reconfile_wk5thJul.csv

file2 = drive.CreateFile({'parents': [{'id': '15w_MeKLpCE2hs8-Vvs4IUBH_sZ8e3qbP'}]})
file2.SetContentFile("reconfile_TestFinal.csv")
file2.Upload()

In [ ]:
# Write recon file to google sheets
# reconSheet = client.open_by_key('1hWr33xeKPsgjA5oRjADCFHtH3CkeiebdJ1eMM6IRia0')
# wksreconfile = reconSheet[1]

# wksreconfile.set_dataframe(reconfile,(1,1))

In [ ]:
#  export payments file to csv
paymentsCombined_csv = combinedPayments.to_csv( "paymentsfile_wk5thJul.csv", index=False, encoding='utf-8-sig')

file2 = drive.CreateFile({'parents': [{'id': '15w_MeKLpCE2hs8-Vvs4IUBH_sZ8e3qbP'}]})
file2.SetContentFile("paymentsfile_wk5thJul.csv")
file2.Upload()

Write reconciled files into google sheets

In [ ]:
# create new sheet for the week

weekname = "May 11"
sh = gc.open_by_key('1Q3BJcXwRncZiiB3GAYw1CNN-RWOWjt4AyUHxjs_Kztg')
weeksheet = sh.add_worksheet(title=weekname, rows="100", cols="10")

In [ ]:
weekperformance = pd.merge(wkperffile,mooveDetails, how='left', on='Name')
weekperformance = weekperformance[['DRN', 'Name',  'Product', 'Net Fares', 'Per trip', 'Per hour online', 'Per km on trip',
       'Trips', 'Hours online', 'Trips per hour', 'Distance per trip',
       'Ratings', 'Lifetime rating', 'Acceptance rate',
       'Driver cancellation rate' ]]
